In [1]:
# weight decay
# undirected
IRT_DIMS = 0
DATASET = 'matrix'    
ITEM_FEATURES = 'True'
ITEM_FEATURES = False if ITEM_FEATURES == 'False' else True

In [2]:
import sys, os, shutil
import pickle
import time
import numpy as np
from utils import (mymode, load_data_heterogeneous, create_data_object_heterogeneous, create_data_object_heterogeneous_temporal)
import seaborn as sns

from IRT import MIRT_2PL
from manage_experiments import perform_cross_validation

import torch
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import pearsonr

In [3]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
# Initialise
parameters = {
    'df_name': None,
    'epochs': 10000,
    'learning_rate': 0.005,
    'weight_decay': 0.0,
    'early_stopping': 200,
    'n_splits': 10,
    'device': 'cuda:0',
    'batch_size': 2**13,
#    'neighbours': [-1, -1]
    'neighbours': [50, 50]
    }

if IRT_DIMS > 0:
    parameters['model_type'] = 'IRT'
    parameters['hidden_dims'] = IRT_DIMS
    parameters['lambda1'] = 0
    parameters['lambda2'] = 0
    OUTNAME = 'IRT'
else:
    parameters['model_type'] = 'GNN'
    parameters['hidden_dims'] = [8,8]
    OUTNAME = 'SAGE' 

if ITEM_FEATURES:
    OUTNAME = OUTNAME + '_scales'
    
print(parameters)
print(DATASET)

{'df_name': None, 'epochs': 10000, 'learning_rate': 0.005, 'weight_decay': 0.0, 'early_stopping': 200, 'n_splits': 10, 'device': 'cuda:0', 'batch_size': 8192, 'neighbours': [50, 50], 'model_type': 'GNN', 'hidden_dims': [8, 8]}
matrix


In [5]:
range(1)

range(0, 1)

In [6]:
DATA_FILE = 'data/mindsteps_set_' + DATASET
df = load_data_heterogeneous(DATA_FILE)

In [7]:
if IRT_DIMS > 0:
    data, df_student, df_item, df_edge = create_data_object_heterogeneous(df, return_aux_data=True, item_features=ITEM_FEATURES)
else:
    data, df_student, df_student_age, df_item, df_edge = create_data_object_heterogeneous_temporal(df, return_aux_data=True, item_features=ITEM_FEATURES)

In [8]:
print(data)

HeteroData(
  student={
    node_id=[636327],
    x=[636327, 4]
  },
  item={
    node_id=[17868],
    x=[17868, 11]
  },
  (student, responds, item)={
    edge_index=[2, 5613692],
    edge_attr=[5613692, 2],
    y=[5613692]
  },
  (item, rev_responds, student)={ edge_index=[2, 5613692] },
  (student, preceeds, student)={ edge_index=[2, 584819] }
)


%load_ext autoreload
%autoreload
from Heterogeneous_temporal_embedder import EmbedderHeterogeneous, train_embedder_heterogeneous, test_embedder_heterogeneous
model = EmbedderHeterogeneous( 
    n_students = data['student'].node_id.size(0),
    n_items = data['item'].node_id.size(0),
    student_inchannel = data['student'].x.size(1),
    item_inchannel = data['item'].x.size(1),
    hidden_channels = parameters['hidden_dims'],
    edge_channel = 2,
    metadata = data.metadata()
    ).to(device)
#model(data)

optimizer = torch.optim.Adam(model.parameters(), lr=parameters['learning_rate'], weight_decay=parameters['weight_decay'])
criterion = torch.nn.BCEWithLogitsLoss().to(device)#class_weights)
loss = []
for i in range(3000):
    l = train_embedder_heterogeneous(model, data, optimizer, criterion)
    loss.append(l)
    if i % 100 == 0:
        print(i, l)
        metrics = test_embedder_heterogeneous(model, data, 0, 'test')
        print(metrics)


train_losses = torch.Tensor(loss).detach().numpy()
train_indices = np.arange(len(train_losses))
fig = plt.figure()
plt.plot(train_indices, train_losses, c='blue')
plt.ylabel('Loss')
plt.xlabel('Epoch')

fig.tight_layout()

In [ ]:
%load_ext autoreload
%autoreload
from manage_experiments import perform_cross_validation
output_dict, model = perform_cross_validation(data, parameters, save_embeddings=True, save_subgraph=True, final_fit=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


0it [00:00, ?it/s]

Parameters
{'n_students': 636327, 'n_items': 17868, 'student_inchannel': 4, 'item_inchannel': 11, 'hidden_channels': [8, 8], 'edge_channel': 2}
[8, 8]



  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

 21%|██        | 16/78 [00:31<02:00,  1.94s/it]

 21%|██        | 16/78 [00:41<02:00,  1.94s/it]

 44%|████▎     | 34/78 [01:02<01:20,  1.82s/it]

 44%|████▎     | 34/78 [01:21<01:20,  1.82s/it]

 67%|██████▋   | 52/78 [01:33<00:46,  1.77s/it]

 67%|██████▋   | 52/78 [01:51<00:46,  1.77s/it]

100%|██████████| 78/78 [02:13<00:00,  1.71s/it]



Epoch: 001, Loss: 0.4660, Val: 0.6007, Test: 0.6054



  0%|          | 1/10000 [02:14<374:15:01, 134.74s/it]

  0%|          | 0/78 [00:00<?, ?it/s]

 26%|██▌       | 20/78 [00:31<01:30,  1.55s/it]

 26%|██▌       | 20/78 [00:46<01:30,  1.55s/it]

 50%|█████     | 39/78 [01:01<01:02,  1.59s/it]

 50%|█████     | 39/78 [01:16<01:02,  1.59s/it]

 68%|██████▊   | 53/78 [01:33<00:46,  1.85s/it]

 68%|██████▊   | 53/78 [01:46<00:46,  1.85s/it]

100%|██████████| 78/78 [02:07<00:00,  1.63s/it]



Epoch: 002, Loss: 0.4257, Val: 0.6755, Test: 0.6785



  0%|          | 2/10000 [04:23<364:02:31, 131.08s/it]

  0%|          | 0/78 [00:00<?, ?it/s]

 26%|██▌       | 20/78 [00:30<01:27,  1.50s/it]

 26%|██▌       | 20/78 [00:48<01:27,  1.50s/it]

 51%|█████▏    | 40/78 [01:00<00:57,  1.50s/it]

 51%|█████▏    | 40/78 [01:18<00:57,  1.50s/it]

 77%|███████▋  | 60/78 [01:31<00:27,  1.53s/it]

100%|██████████| 78/78 [01:56<00:00,  1.49s/it]



Epoch: 003, Loss: 0.4007, Val: 0.7018, Test: 0.7035



  0%|          | 3/10000 [06:20<346:40:58, 124.84s/it]

  0%|          | 0/78 [00:00<?, ?it/s]

 26%|██▌       | 20/78 [00:31<01:30,  1.57s/it]

 26%|██▌       | 20/78 [00:50<01:30,  1.57s/it]

 51%|█████▏    | 40/78 [01:03<01:00,  1.58s/it]

 51%|█████▏    | 40/78 [01:20<01:00,  1.58s/it]

In [ ]:
output_dict.keys()
print('AUC:', output_dict['AUC_0_test'])
print('Balanced Accuracy:', output_dict['Balanced Accuracy_0_test'])

In [ ]:
unique_scales = df_item['scale'].unique()
unique_domains = df_item['domain'].unique()
unique_matdiff = df_item['matdiff'].sort_values().unique()
#scale_colors = dict([(c, plt.cm.tab10(i)) for i, c in enumerate(unique_scales)])
#domain_colors = dict([(c, plt.cm.tab10(i)) for i, c in enumerate(unique_domains)])

In [ ]:
train_losses = output_dict['losses_0']
train_edge_indices, val_edge_indices, test_edge_indices = output_dict['indices_0']

train_data = output_dict['train_subgraph_data'] 
val_data = output_dict['val_subgraph_data'] 
test_data = output_dict['test_subgraph_data']

aux_data = (df, df_student, df_item, df_edge, 
    #clustering_indices, 
    train_losses, #test_losses, test_aucs, 
    train_edge_indices, val_edge_indices, test_edge_indices, 
    data.cpu(), train_data.cpu(), val_data.cpu(), test_data.cpu(),
    unique_scales, unique_domains, unique_matdiff, 
    DATA_FILE)

with open(f'./results/{OUTNAME}_{DATASET}_aux_data.pkl', 'wb') as handle:
    pickle.dump(aux_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# SAVE EVERYTHING
torch.save(model, f'./results/{OUTNAME}_{DATASET}.pth.tar')

In [ ]:
train_indices = np.arange(len(train_losses))
fig = plt.figure()
plt.plot(train_indices, train_losses, c='blue')
plt.ylabel('Loss')
plt.xlabel('Epoch')

fig.tight_layout()

In [ ]:
# save IRT parameters to matrix
if OUTNAME == 'IRT' and IRT_DIMS == 1:    
    #z_dict = model.get_embeddings(train_data.to(device))
    z_dict = output_dict['embedding_0']
    df_item['IRT1_difficulty'] = -z_dict['offset']
    df_item['IRT1_discrimination'] = z_dict['discrimination']
    df_item['IRT1_discrimination_transf'] = z_dict['item']
    ability = z_dict['ability']
    df_edge['IRT1_ability'] = ability.ravel()
    
    aux_data = (df, df_student, df_item, df_edge, 
    train_losses, 
    train_edge_indices, val_edge_indices, test_edge_indices,
    data.cpu(), train_data.cpu(), val_data.cpu().cpu(), test_data.cpu(),
    unique_scales, unique_domains, unique_matdiff, 
    DATA_FILE)

    with open(f'./results/{OUTNAME}_{DATASET}_aux_data_IRT1.pkl', 'wb') as handle:
        pickle.dump(aux_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    df_item_clean = df_item.dropna(subset=['IRT_difficulty', 'IRT1_difficulty'])    
    
    fig = plt.figure()
    sns.scatterplot(x='IRT_difficulty', y='IRT1_difficulty', data=df_item, hue='scale')
    plt.title('Difficulty')
    print('Difficulty:', pearsonr(df_item_clean['IRT1_difficulty'], df_item_clean['IRT_difficulty']))
    
    edge_feat = train_data['student', 'responds', 'item'].edge_attr.detach().cpu().numpy()
    df_edge_clean = df_edge.dropna(subset=['IRT1_ability', 'ability', 'age'])
    fig = plt.figure()
    sns.scatterplot(x='age', y='IRT1_ability', data=df_edge_clean, hue='grade')
    plt.title('Age-Ability')
    
    fig = plt.figure()
    sns.scatterplot(x='grade', y='IRT1_ability', data=df_edge_clean, hue='age')
    plt.title('Grade-Ability')
    print('Age-Ability:', pearsonr(df_edge_clean['age'], df_edge_clean['IRT1_ability']))

    fig = plt.figure()
    sns.scatterplot(x='ability', y='IRT1_ability', data=df_edge_clean, hue='grade')
    plt.title('Ability')
    print('Ability:', pearsonr(df_edge_clean['ability'], df_edge_clean['IRT1_ability']))
    